In [1]:
import os
os.chdir("..")

In [2]:
from util.google_search_util import google_the_claim

In [3]:
from selenium import webdriver

In [4]:
import requests
from bs4 import BeautifulSoup

In [5]:
claim = "Brad Pitt was married to Angelina Jolie"

In [6]:
claim = "Arnold acted in Conan"

In [14]:
claim = "Black Mirror started airing in 2011."

In [5]:
claim = "The United States has over 300 million people."

In [6]:
google_query = "+".join(claim.split(" "))
page_url = f"https://www.google.com/search?q={google_query}"

In [7]:
page_url

'https://www.google.com/search?q=The+United+States+has+over+300+million+people.'

In [17]:
page = requests.get(page_url)
page.raise_for_status()
soup = BeautifulSoup(page.content, "html.parser")

HTTPError: 429 Client Error: Too Many Requests for url: https://www.google.com/sorry/index?continue=https://www.google.com/search%3Fq%3DBlack%2BMirror%2Bstarted%2Bairing%2Bin%2B2011.&q=EgStLDApGLTcr-cFIhkA8aeDS_ZS5GiKcpotO9EwlMWMzZEDueXMMgFy

In [8]:
from selenium.webdriver import ChromeOptions

In [9]:
options = ChromeOptions()

In [10]:
options.add_argument("--headless")

In [11]:
browser = webdriver.Chrome(executable_path='./chromedriver', options = options)

In [20]:
browser = webdriver.Chrome(executable_path='./chromedriver')

In [12]:
browser.get(page_url)
soup = BeautifulSoup(browser.page_source, 'html.parser')

In [13]:
rcs = soup.find_all("div", class_="rc")

In [14]:
titles = [rc.find("div", class_="r").find("h3", class_="LC20lb").text.replace(u'\xa0', u' ') for rc in rcs]

In [15]:
titles

['The United States at 300 Million – Population Reference Bureau',
 'What Is the Population of the United States? - ThoughtCo',
 'United States Population 2019 (Demographics, Maps, Graphs)',
 'This Is How Fast the U.S. Population Is Growing | Fortune',
 "Does the United States really have 5 percent of the world's ...",
 'U.S. population now 300 million and growing - CNN.com',
 'Population Clock - Census Bureau',
 'America at 300 Million - Brookings Institution',
 'United States - Worldometers',
 '300 Million and Counting | History | Smithsonian',
 'Demography of the United States - Wikipedia',
 'Which is higher: The number of people, or the number of guns, in ...',
 'US City Populations 2019 - World Population Review']

In [30]:
rcs[1]

<div class="rc"><div class="r"><a href="https://en.m.wikipedia.org/wiki/The_National_Anthem_(Black_Mirror)" ping="/url?sa=t&amp;source=web&amp;rct=j&amp;url=https://en.m.wikipedia.org/wiki/The_National_Anthem_(Black_Mirror)&amp;ved=2ahUKEwib6o7d8LviAhVO1lkKHcj-C8IQFjAEegQIDxAN"><h3 class="LC20lb">The National Anthem (Black Mirror) - Wikipedia</h3><br/><div class="TbwUpd"><cite class="iUh30">https://en.m.wikipedia.org/wiki/The_National_Anthem_(Black_Mirror)</cite></div></a></div><div class="s"><div></div></div></div>

In [17]:
[rc.find("div", class_="s").find("span", class_="st").text if rc.find("div", class_="s") and rc.find("div", class_="s").find("span", class_="st") else ""  for rc in rcs]

['Sep 1, 2006 - Each 100 million has been added more quickly than the last. It took the United States more than 100 years to reach its first 100 million in 1915.',
 'Jan 20, 2019 - The current U.S. population is more than 300 million people as of early 2018 and expected to be near 420 million by the year 2060.',
 '',
 '',
 '',
 'Oct 17, 2006 - The United States became a nation of 300 million people Tuesday morning, ... joined China and India with populations greater than 300 million.',
 'U.S. Population. The U.S. population clock is based on a series of short-term projections for the resident population of the United States. This includes people\xa0...',
 'Oct 8, 2006 - Today the United States is 50% larger in population and development footprint ... versus about 5% when the country had 200 million people.',
 'The current population of the United States of America is 328,862,603 as of ... The population density in the United States is 36 per Km2 (93 people per mi2).',
 "The United Stat

In [24]:
summaries = [rc.find("div", class_="s").find("span", class_="st").text.replace(u'\xa0', u' ') for rc in rcs]

AttributeError: 'NoneType' object has no attribute 'text'

In [25]:
titles

['Black Mirror - Wikipedia',
 'The National Anthem (Black Mirror) - Wikipedia',
 "Black Mirror's nightmarish social media episode is now a board ...",
 'List of Black Mirror episodes - Wikipedia',
 'Nosedive (Black Mirror) - Wikipedia',
 'Black Mirror (TV Series 2011– ) - IMDb',
 'Black Mirror | Netflix Official Site',
 'Black Mirror | Black Mirror Wiki | FANDOM powered by Wikia',
 "It's official: Netflix is picking up Black Mirror's third season - The Verge",
 'Netflix will be releasing new episodes of Black Mirror on October 21st ...',
 'Black Mirror mini-episodes arrive on YouTube starting May 26 ...',
 'With “Black Mirror,” Our Dystopia Gets the Television Show It Deserves ...']

In [49]:
titles_no_page = [t.split(" - ", 1)[0] for t in titles]

In [50]:
titles_no_page

['Conan the Barbarian (1982 film)',
 'Conan the Barbarian (1982)',
 "Arnold Schwarzenegger Is Still Trying to Get 'Legend of Conan' Made ...",
 "Arnold Schwarzenegger Still Wants to Do Another 'Conan the ...",
 'Conan (Arnold Schwarzenegger) | Conan Wiki | FANDOM powered by ...',
 "'Conan the Barbarian' at 35: Arnold Schwarzenegger's Path to Fame ...",
 "Arnold Schwarzenegger Explains Legend of Conan's Development ...",
 'Arnold Schwarzenegger Still Wants to Make a King Conan Movie']

In [82]:
titles_no_page_no_dots = [t[:-4] if t.endswith("...") else t for t in titles_no_page]

In [83]:
titles_no_page_no_dots

['Conan the Barbarian (1982 film)',
 'Conan the Barbarian (1982)',
 "Arnold Schwarzenegger Is Still Trying to Get 'Legend of Conan' Made",
 "Arnold Schwarzenegger Still Wants to Do Another 'Conan the",
 'Conan (Arnold Schwarzenegger) | Conan Wiki | FANDOM powered by',
 "'Conan the Barbarian' at 35: Arnold Schwarzenegger's Path to Fame",
 "Arnold Schwarzenegger Explains Legend of Conan's Development",
 'Arnold Schwarzenegger Still Wants to Make a King Conan Movie']

In [59]:
summaries

['Conan the Barbarian is a 1982 American fantasy adventure film directed and co-written by .... Her sacrifice for Conan and her brief return from death act out the heroic code, illustrating that self-sacrificing heroism brings "undying fame". Valeria\'s ...',
 'Directed by John Milius. With Arnold Schwarzenegger, James Earl Jones, Max von Sydow, Sandahl Bergman. A young boy, Conan, becomes a slave after his ...',
 "15. velj 2019. - The Conan actor has shown significant interest in reprising his most iconic roles since he returned to acting following his tenure as California's ...",
 "8. ožu 2019. - Following the reboot which starred Jason Momoa, Warner Bros announced ... Arnold Still Wants to Make Another 'Conan the Barbarian' Movie ...",
 'Summer envisioned a series of Conan films, much like the James Bond franchise. ... The character of Conan was played by Arnold Schwarzenegger and was his ...',
 "13. svi 2017. - 'Conan the Barbarian' at 35: How Darth Vader Helped Arnold ... Schwarze

In [62]:
summaries_no_date = [s.split(" - ", 1)[-1] for s in summaries]

In [63]:
summaries_no_date

['Conan the Barbarian is a 1982 American fantasy adventure film directed and co-written by .... Her sacrifice for Conan and her brief return from death act out the heroic code, illustrating that self-sacrificing heroism brings "undying fame". Valeria\'s ...',
 'Directed by John Milius. With Arnold Schwarzenegger, James Earl Jones, Max von Sydow, Sandahl Bergman. A young boy, Conan, becomes a slave after his ...',
 "The Conan actor has shown significant interest in reprising his most iconic roles since he returned to acting following his tenure as California's ...",
 "Following the reboot which starred Jason Momoa, Warner Bros announced ... Arnold Still Wants to Make Another 'Conan the Barbarian' Movie ...",
 'Summer envisioned a series of Conan films, much like the James Bond franchise. ... The character of Conan was played by Arnold Schwarzenegger and was his ...',
 "'Conan the Barbarian' at 35: How Darth Vader Helped Arnold ... Schwarzenegger even served as the Governor of California

In [79]:
summaries_no_date_no_dots = [s.replace(" ... ", ", ").replace(" ...", "") for s in summaries_no_date]

In [80]:
summaries_no_date_no_dots

['Conan the Barbarian is a 1982 American fantasy adventure film directed and co-written by. Her sacrifice for Conan and her brief return from death act out the heroic code, illustrating that self-sacrificing heroism brings "undying fame". Valeria\'s',
 'Directed by John Milius. With Arnold Schwarzenegger, James Earl Jones, Max von Sydow, Sandahl Bergman. A young boy, Conan, becomes a slave after his',
 "The Conan actor has shown significant interest in reprising his most iconic roles since he returned to acting following his tenure as California's",
 "Following the reboot which starred Jason Momoa, Warner Bros announced, Arnold Still Wants to Make Another 'Conan the Barbarian' Movie",
 'Summer envisioned a series of Conan films, much like the James Bond franchise., The character of Conan was played by Arnold Schwarzenegger and was his',
 "'Conan the Barbarian' at 35: How Darth Vader Helped Arnold, Schwarzenegger even served as the Governor of California from 2003 to 2011.",
 'After yea

In [89]:
result = [f"{t} {s}"  for t, s in zip(titles_no_page_no_dots, summaries_no_date_no_dots)]

In [94]:
final_result = " ".join(result)

In [2]:
from util.google_search_util import google_the_claim_text_results

In [4]:
google_the_claim_text_results("Kristijan Palic likes to eat shit.")

"Biotechnology Ružičić, Stanko; Mileusnić, Marta; Posavec, Kristijan; Nakić, Zoran; Durn,, Beer-Ljubić, Blanka; Matijatko, Vesna; Palić, Dušan; Klobučar, Goran; Babić,... Non-dioxin-like PCBs in ten different fish species from the Danube river in Serbia.... Production characteristics and health status of calves fed diff erent amounts of njuz | najawa | koncert || ... || kontakt | search Eat shit.... Zagreb · [najawa] Trenchtown festival @ Etno kamp Palić (SR), 30.04. Possible cause of unnatural mass death of wild birds in a pond in Aage Kristian Olsen Alstrup.. selective alimentation eliminating living Microcystis in pseudo-feces to avoid microcystin producers (Vanderploeg et al., 2001). Colorz » Where is your De Coubertin now? palić. Palacze w tym samym stopniu decydują, iż chcą stanowić nałogowcami, w jakim opój decyduje się, że chce utrzymywać się.. 64 • Kristian • 5 novembre 2013 alle 15:20. Do You... Hmm it appears like your blog ate my first comment (it was super... Because I do,

In [ ]:
"BNeawe vvjwJb AP7Wnd"

In [16]:
[t for t in soup.find_all("div", class_="BNeawe vvjwJb AP7Wnd")]

[]

In [12]:
[t.text.split("·", 1)[-1].replace(u'\xa0', u' ') for t in soup.find_all("div", class_="BNeawe s3v9rd AP7Wnd") if t.div]

['Conan Cimmerijanac jest izmišljeni lik koji se često dovodi u vezu sa podžanrom low fantasy, odnosno mač i čarobnjaštvo, koji je također poznat pod nazivom herojska fantastika. Smatra se najpoznatijim fiktivnim barbarom i uopće jednom od... Wikipedia',
 ' Conan the Barbarian 1982 A barbarian warrior sets off to avenge his parents and his tribe ...Trajanje: 3:21\nObjavljeno: 7. lip 2017.',
 ' Production · Music · Reception\n\nMusic by: Basil Poledouris\nBudget: $16 million\nProduced by: Buzz Feitshans; Raffaella De Laurentiis\nRelease date: March 16, 1982 (Spain); May 14, 1982 (North America)',
 'Ocjena  6,9/10  (128.002) Directed by John Milius. With Arnold Schwarzenegger, James Earl Jones, Max von Sydow, Sandahl Bergman. A young boy, Conan, becomes a slave after his ...',
 'Ocjena  6,9/10  (128.002) ',
 " The Conan actor has shown significant interest in reprising his most iconic roles since he returned to acting following his tenure as California's ...",
 " Following the reboot whi

In [21]:
google_the_claim("Shes the boss", verbose=True, starting_from_page=0, search_engine="yahoo")

https://search.yahoo.com/search?p=Shes+the+boss&b=0
Found url: https://en.wikipedia.org/wiki/She%27s_the_Boss
Found url: https://video.search.yahoo.com/search/video?p=Shes+the+boss
Found url: https://www.youtube.com/watch?v=Fa2fGMsBOr0
Found url: https://www.rollingstone.com/music/music-album-reviews/shes-the-boss-94031/#!
Found url: https://images.search.yahoo.com/search/images?p=Shes+the+boss
Found url: https://shes-the-boss.net/
Found url: https://www.imdb.com/title/tt6381432/
Found url: https://www.amazon.com/Shes-Boss-MICK-JAGGER/dp/B000002IXG
Found url: https://www.wattpad.com/424559669-she%27s-the-boss-5
Found url: https://www.wattpad.com/423479455-she%27s-the-boss-2
Found url: https://www.allmusic.com/album/shes-the-boss-mw0000190458
Found url: https://www.cbs.com/shows/frasier/video/t59emfV7rPi1G47_S3W0HXt0uQLp7neX/frasier-she-s-the-boss/


["She's_the_Boss"]

In [3]:
google_the_claim("Shes the boss", verbose=True, starting_from_page=0)

https://www.google.com/search?q=Shes+the+boss&start=0
Found url: https://en.wikipedia.org/wiki/She%2527s_the_Boss
Found url: https://www.youtube.com/watch%3Fv%3DFa2fGMsBOr0
Found url: https://www.youtube.com/watch%3Fv%3DfrQLxn0uOD8
Found url: https://www.discogs.com/Mick-Jagger-Shes-The-Boss/master/97173
Found url: https://www.rollingstone.com/music/music-album-reviews/shes-the-boss-94031/
Found url: https://www.azlyrics.com/lyrics/mickjagger/shestheboss.html
Found url: https://www.amazon.com/Shes-Boss-MICK-JAGGER/dp/B000002IXG
Found url: https://www.allmusic.com/album/shes-the-boss-mw0000190458
Found url: https://genius.com/Mick-jagger-shes-the-boss-lyrics
Found url: https://open.spotify.com/album/4AXZg2u76JOqk2gV1iKH3x


["She's_the_Boss"]

In [4]:
google_the_claim("Shes the boss", verbose=True, starting_from_page=1)

https://www.google.com/search?q=Shes+the+boss&start=10
Found url: https://www.karmavinil.com/jagger-mick-shes-the-boss-49401/
Found url: https://www.eventbrite.com/e/shes-the-bosstm-events-march-presents-show-me-the-money-tickets-57041608988
Found url: https://lmarienaillacquer.com/products/sample-product/shes-the-boss/
Found url: https://www.nytimes.com/1985/01/28/arts/jagger-finishes-first-solo-album-she-s-the-boss.html
Found url: https://pantherecords.com/index.php/product/mick-jagger-shes-the-boss-sealed/
Found url: https://www.imdb.com/title/tt1485781/


[]

In [5]:
google_the_claim("Shes the boss", verbose=True, starting_from_page=2)

https://www.google.com/search?q=Shes+the+boss&start=20
Found url: https://www.amazon.co.uk/Shes-Boss-Mick-Jagger/dp/B000002IXG
Found url: https://www.facebook.com/pages/category/House-Painting/Shes-the-Boss-Painting-777138135724315/
Found url: https://www.quicket.co.za/events/67368-shes-the-boss/
Found url: https://www.goodreads.com/book/show/16198045-she-s-the-boss
Found url: https://www.stitcher.com/podcast/shes-the-boss-podcast/shes-a-boss-podcast
Found url: https://www.nytimes.com/1985/01/28/arts/jagger-finishes-first-solo-album-she-s-the-boss.html
Found url: https://www.racingpost.com/profile/horse/864554/shes-the-boss
Found url: https://quickbooks.intuit.com/blog/thought-leadership/shes-the-boss-mistakes-learned-from-working-with-my-husband/
Found url: https://yamileburich.bandcamp.com/album/shes-the-boss-2
Found url: https://lareviewofbooks.org/article/shes-the-boss/


[]

In [6]:
google_the_claim("Shes the boss", verbose=True, pages_to_check=3)

https://www.google.com/search?q=Shes+the+boss&start=0
Found url: https://en.wikipedia.org/wiki/She%2527s_the_Boss
Found url: https://www.youtube.com/watch%3Fv%3DFa2fGMsBOr0
Found url: https://www.youtube.com/watch%3Fv%3DfrQLxn0uOD8
Found url: https://www.youtube.com/watch%3Fv%3DKqoXN0cLRs0
Found url: https://www.discogs.com/Mick-Jagger-Shes-The-Boss/master/97173
Found url: https://www.discogs.com/Mick-Jagger-Shes-The-Boss/release/765807
Found url: https://www.rollingstone.com/music/music-album-reviews/shes-the-boss-94031/
Found url: https://www.azlyrics.com/lyrics/mickjagger/shestheboss.html
Found url: https://www.amazon.com/Shes-Boss-MICK-JAGGER/dp/B000002IXG
Found url: https://www.allmusic.com/album/shes-the-boss-mw0000190458
https://www.google.com/search?q=Shes+the+boss&start=10
Found url: https://genius.com/Mick-jagger-shes-the-boss-lyrics
Found url: https://www.imdb.com/title/tt6381432/
Found url: https://open.spotify.com/album/4AXZg2u76JOqk2gV1iKH3x
Found url: https://www.legit.n

["She's_the_Boss"]

In [5]:
google_the_claim("Arnold acted in Conan", limit_to_wiki=True)

['Conan_the_Barbarian_(1982_film)',
 'Arnold_Schwarzenegger_filmography',
 'Conan_the_Barbarian_(2011_film)',
 'Sven-Ole_Thorsen',
 'Conan_the_Destroyer',
 'Arnold_Schwarzenegger',
 'Conan_the_Barbarian',
 'Conan_the_Adventurer_(1997_TV_series)',
 'Sandahl_Bergman']

In [4]:
google_the_claim("Arnold acted in Conan", limit_to_wiki=True, search_engine="yahoo")

['Conan_the_Barbarian_(1982_film)',
 'Arnold_Schwarzenegger_filmography',
 'Conan_the_Barbarian_(2011_film)',
 'Red_Sonja_(1985_film)',
 'Sherlock_(TV_series)',
 'Grace_Jones',
 'The_Terminator',
 'Talk:Conan_the_Barbarian_(1982_film)',
 'Twins_(1988_film)',
 'List_of_comedians']

In [5]:
google_the_claim("Luis Armstrong won the superbowl.")

['Super_Bowl_VI', 'Louis_Armstrong']

In [7]:
google_the_claim("Luis Armstrong won the superbowl.", search_engine="yahoo", verbose=True)

https://search.yahoo.com/search?p=Luis+Armstrong+won+the+superbowl.&start=0
Found url: https://www.youtube.com/watch?v=wyLjbMBpGDA
Found url: https://www.cbsnews.com/news/super-bowl-ad-winners-and-losers/
Found url: https://www.si.com/nfl/2018/01/21/what-teams-have-never-won-super-bowl
Found url: https://www.aol.com/article/news/2018/01/30/13-nfl-teams-that-have-never-won-a-super-bowl/23347992/
Found url: https://www.sbnation.com/2018/2/5/16974748/how-many-nfl-teams-without-super-bowl-win
Found url: https://www.forbes.com/sites/robsalkowitz/2017/02/05/whos-winning-commercial-bowl-keep-score-here-during-the-big-game/
Found url: https://www.cbssports.com/nfl/news/after-eagles-win-these-are-the-only-12-teams-to-have-never-won-a-super-bowl/
Found url: https://www.cbsnews.com/news/philadelphia-eagles-win-super-bowl-lii-nick-foles-mvp-2018/
Found url: https://www.aol.com/article/sports/2017/02/03/the-13-nfl-teams-that-have-never-won-a-super-bowl/21706689/
Found url: https://www.cbssports.com

[]

In [5]:
google_the_claim("Luis Armstrong won the superbowl.", search_engine="yahoo", limit_to_wiki=True, starting_from_page=1, verbose=True)

https://search.yahoo.com/search?p=Luis+Armstrong+won+the+superbowl.+site%3Aen.wikipedia.org&b=11
Found url: https://en.wikipedia.org/wiki/Marcus_Allen
Found url: https://en.wikipedia.org/wiki/Desmond_Howard
Found url: https://en.wikipedia.org/wiki/List_of_The_Ring_world_champions
Found url: https://en.wikipedia.org/wiki/Carson_Wentz
Found url: https://en.wikipedia.org/wiki/William_Perry_(American_football)
Found url: https://en.wikipedia.org/wiki/Eddie_George
Found url: https://en.wikipedia.org/wiki/List_of_Olympic_medalists_in_athletics_(men)
Found url: https://en.wikipedia.org/wiki/Colorado_Rapids
Found url: https://en.wikipedia.org/wiki/Reggie_White
Found url: https://en.wikipedia.org/wiki/List_of_current_NFC_team_rosters


['Marcus_Allen',
 'Desmond_Howard',
 'List_of_The_Ring_world_champions',
 'Carson_Wentz',
 'William_Perry_(American_football)',
 'Eddie_George',
 'List_of_Olympic_medalists_in_athletics_(men)',
 'Colorado_Rapids',
 'Reggie_White',
 'List_of_current_NFC_team_rosters']

In [6]:
google_the_claim("Luis Armstrong won the superbowl.", limit_to_wiki=True)

['Super_Bowl_VI',
 'Louis_Armstrong',
 'Tom_Brady',
 '2011_New_York_Giants_season',
 'Super_Bowl_XXXVII',
 'Joe_Montana',
 'Drew_Brees',
 'Deion_Sanders',
 'Miami_Dolphins']

In [7]:
google_the_claim("Leo messi won an oscar")

['Lionel_Messi']

In [8]:
google_the_claim("Marin Sokol won an oscar", verbose=True)

Found url: https://en.wikipedia.org/wiki/List_of_Japanese_submissions_for_the_Academy_Award_for_Best_Foreign_Language_Film
Found url: https://en.wikipedia.org/wiki/C%25C3%25A9sar_Award_for_Best_Actress
Found url: https://en.wikipedia.org/wiki/Roman_Polanski
Found url: https://www.imdb.com/title/tt0338751/
Found url: https://www.imdb.com/name/nm1046789/
Found url: https://www.imdb.com/title/tt0295545/
Found url: https://www.nature.com/srep/about/editorial-board
Found url: https://www.realert.com/article_list.pl
Found url: https://bayareacabaret.org/pr_past.html


['List_of_Japanese_submissions_for_the_Academy_Award_for_Best_Foreign_Language_Film',
 'César_Award_for_Best_Actress',
 'Roman_Polanski']

In [9]:
google_the_claim("Marin Sokol won an oscar", limit_to_wiki=True)

['Roman_Polanski',
 '1972_NBA_All-Star_Game',
 'List_of_Olympic_medalists_in_volleyball',
 'Italian_training_ship_Corsaro_II',
 'USS_Slater',
 'HMS_Cavalier_(R73)',
 'List_of_Juilliard_School_people',
 'Elena_Risteska',
 'USS_McAnn']

In [10]:
google_the_claim("Égalité")

['Liberté,_égalité,_fraternité']

In [11]:
google_the_claim("Über")

['Über']

In [12]:
google_the_claim("Rosendals Trädgård was great")

['Rosendals_Trädgård']

In [13]:
google_the_claim("Marin držić")

['Marin_Držić', 'Marin_Držić']

In [14]:
google_the_claim("game of thrones")

['Game_of_Thrones']

In [15]:
google_the_claim("igra prijestolja")

['Game_of_Thrones']